# Background Work for Part 1 and Part 2 Week 4 assignment, "The Battle of the Neighborhoods"
Starting off with code for pulling in data sources and clustering them together before going into the report.  Thanks for the patience!

In [47]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from unicodedata import normalize
!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
from IPython.display import Image 
from IPython.core.display import HTML
! pip install folium==0.5.0
import folium # plotting library

print('Libraries imported.')

Libraries imported.


In [48]:
# Go get Louisiana Metropolitan service area data from wikipedia to begin pulling in the data.
table_LA = pd.read_html('https://www.louisiana-demographics.com/cities_by_population', match='City')

In [49]:
# Got the idea for pd.read_html from https://pbpython.com/pandas-html-table.html, great tool!
print(f'Total tables: {len(table_LA)}')
dfLA= table_LA[0]


Total tables: 1


In [50]:
dfLA.head(8)
# Show the top 8 cities by population in Louisiana

,Rank,City,Population
0,1,New Orleans,390144
1,2,Baton Rouge,220236
2,3,Shreveport,187112
3,4,Metairie,143481
4,5,Lafayette,126185
5,6,Lake Charles,78396
6,7,Bossier City,68159
7,8,Kenner,66340


In [51]:
dfLAtop8=dfLA.head(8)
# Shrink list to only pull latitude/longitude data for the top 8 cities.


In [52]:
# https://www.geeksforgeeks.org/how-to-find-longitude-and-latitude-for-a-list-of-regions-or-country-using-python/

! pip install geopandas
! pip install geopy
from geopy.exc import GeocoderTimedOut


In [53]:
# Set up empty list to store latitude and longitude values
longitude=[]
latitude=[]

# set up a function to find the coordinates of the cities in the dataframe
def findGeocode(city):
    # try and catch is used to overcome any exceptions due to a timeout in geocdoer
    try:
        geolocator = Nominatim(user_agent="LA-agent")
        return geolocator.geocode(city)
    except GeocoderTimedOut:
        return findGeocode(city)


In [54]:
# each value from city column will be fetched to and sent to the findGeocode function
for i in (dfLAtop8['City']):
    if findGeocode(i) != None:
        loc = findGeocode(i)
        # build two lists for returned coordinates
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)
    # if coordinates not found, inset "NaN"
    else:
        latitude.append(np.nan)
        longitude.append(np.nan)

In [55]:
# add new columns to the dfLAtop10 dataframe
dfLAtop8['Latitude']=latitude
dfLAtop8['Longitude']=longitude
dfLAtop8.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,Rank,City,Population,Latitude,Longitude
0,1,New Orleans,390144,29.949932,-90.070116
1,2,Baton Rouge,220236,30.445960,-91.187380
2,3,Shreveport,187112,32.522183,-93.765194
3,4,Metairie,143481,29.989993,-90.150543
4,5,Lafayette,126185,30.224090,-92.019843


In [56]:
# create map of Louisiana using latitude and longitude values, center on LeCompte, LA.
address = 'Lecompte, LA'
geolocator = Nominatim(user_agent="LA-agent")
location = geolocator.geocode(address)
lat=location.latitude
long=location.longitude

LA_map = folium.Map(location=[lat, long], zoom_start=8)

# add markers to map
for lat, lng, label in zip(dfLAtop8['Latitude'], dfLAtop8['Longitude'], dfLAtop8['City']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(LA_map)  
    
LA_map

In [57]:
CLIENT_ID = 'A0N2RZ3K4L32FICCEB4X5K5N3BL1REACSXRGT1P2GC2ZSI1J' # your Foursquare ID
CLIENT_SECRET = 'CREJVTS2OI3EJFWIDTGHYN0CAZIQLINOA4XK0QOK2AO4SELG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
# type your answer here


Your credentails:
CLIENT_ID: A0N2RZ3K4L32FICCEB4X5K5N3BL1REACSXRGT1P2GC2ZSI1J
CLIENT_SECRET:CREJVTS2OI3EJFWIDTGHYN0CAZIQLINOA4XK0QOK2AO4SELG


In [58]:
def getNearbyVenues(names, latitudes, longitudes, radius=2500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [59]:
LA_venues = getNearbyVenues(names=dfLAtop8['City'],
                                   latitudes=dfLAtop8['Latitude'],
                                   longitudes=dfLAtop8['Longitude']
                                  )

New Orleans
Baton Rouge
Shreveport
Metairie
Lafayette
Lake Charles
Bossier City
Kenner


In [60]:
LA_venues.groupby('City').count()

,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
Baton Rouge,65,65,65,65,65,65
Bossier City,100,100,100,100,100,100
Kenner,76,76,76,76,76,76
Lafayette,100,100,100,100,100,100
Lake Charles,44,44,44,44,44,44
Metairie,100,100,100,100,100,100
New Orleans,100,100,100,100,100,100
Shreveport,86,86,86,86,86,86


In [61]:
print('There are {} unique categories.'.format(len(LA_venues['Venue Category'].unique())))

There are 172 unique categories.


In [62]:
# one hot encoding
LA_onehot = pd.get_dummies(LA_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
LA_onehot['City'] = LA_venues['City'] 

# move neighborhood column to the first column
fixed_columns = [LA_onehot.columns[-1]] + list(LA_onehot.columns[:-1])
LA_onehot = LA_onehot[fixed_columns]

LA_onehot.head()

,City,Accessories Store,Adult Boutique,Advertising Agency,Airport,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Beach,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Casino,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Baseball Diamond,College Basketball Court,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Food,Food Court,French Restaurant,Fried Chicken Joint,Gaming Cafe,Gas Station,Gastropub,Gay Bar,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Historic Site,History Museum,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Optical Shop,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Service,Pharmacy,Pizza Place,Playground,Plaza,Post Office,Pub,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tex-Mex Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Track,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Weight Loss Center,Wine Shop,Wings Joint
0,New Orleans,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,New Orleans,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,New Orleans,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,New Orleans,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,New Orleans,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [63]:
LA_grouped = LA_onehot.groupby('City').mean().reset_index()
LA_grouped

,City,Accessories Store,Adult Boutique,Advertising Agency,Airport,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Beach,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Casino,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Baseball Diamond,College Basketball Court,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Food,Food Court,French Restaurant,Fried Chicken Joint,Gaming Cafe,Gas Station,Gastropub,Gay Bar,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Historic Site,History Museum,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Optical Shop,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Service,Pharmacy,Pizza Place,Playground,Plaza,Post Office,Pub,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tex-Mex Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Track,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Weight Loss Center,Wine Shop,Wings Joint
0,Baton Rouge,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046154,0.015385,0.00,0.015385,0.015385,0.000000,0.00,0.015385,0.000000,0.000000,0.061538,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.015385,0.015385,0.00,0.00,0.015385,0.000000,0.00,0.00,0.000000,0.015385,0.00,0.00,0.00000,0.000000,0.015385,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.030769,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015385,0.00,0.00,0.00,0.015385,0.000000,0.015385,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.015385,0.000000,0.015385,0.00,0.000000,0.00,0.000000,0.00,0.030769,0.030769,0.000000,0.00,0.000000,0.046154,0.00,0.00,0.00,0.015385,0.015385,0.000000,0.00,0.00,0.00,0.000000,0.00,0.015385,0.000000,0.000000,0.00,0.000000,0.015385,0.015385,0.015385,0.00,0.000000,0.00,0.015385,0.015385,0.00,0.00,0.000000,0.061538,0.030769,0.015385,0.015385,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.030769,0.030769,0.000000,0.015385,0.015385,0.015385,0.000000,0.000000,0.015385,0.000000,0.015385,0.015385,0.015385,0.00,0.015385,0.000000,0.00,0.000000,0.015385,0.015385,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.015385,0.015385,0.015385,0.015385,0.00,0.00,0.015385,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00
1,Bossier City,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.01,0.000000,0.020000,0.020000,0.000000,0.000000,0.00,0.00,0.01

In [64]:
num_top_venues = 5

for city in LA_grouped['City']:
    print("----"+city+"----")
    temp = LA_grouped[LA_grouped['City'] == city].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Baton Rouge----
                 venue  freq
0                  Bar  0.06
1               Museum  0.06
2  American Restaurant  0.05
3                Hotel  0.05
4    Convenience Store  0.03


----Bossier City----
                  venue  freq
0   Fried Chicken Joint  0.06
1    Mexican Restaurant  0.06
2        Clothing Store  0.05
3  Fast Food Restaurant  0.04
4          Burger Joint  0.04


----Kenner----
                  venue  freq
0           Coffee Shop  0.11
1                   Bar  0.05
2  Fast Food Restaurant  0.05
3           Pizza Place  0.05
4    Seafood Restaurant  0.05


----Lafayette----
                       venue  freq
0             Sandwich Place  0.08
1  Cajun / Creole Restaurant  0.06
2                        Bar  0.06
3       Fast Food Restaurant  0.05
4                       Café  0.04


----Lake Charles----
                  venue  freq
0   American Restaurant  0.07
1                Museum  0.05
2   Fried Chicken Joint  0.05
3                   Bar  0.05
4  

In [65]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['City'] = LA_grouped['City']

for ind in np.arange(LA_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(LA_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted.head(8)

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Baton Rouge,Museum,Bar,Hotel,American Restaurant,Historic Site,Park,Performing Arts Venue,Music Venue,History Museum,Convenience Store
1,Bossier City,Fried Chicken Joint,Mexican Restaurant,Clothing Store,Burger Joint,Fast Food Restaurant,Breakfast Spot,Steakhouse,Department Store,Discount Store,Japanese Restaurant
2,Kenner,Coffee Shop,Seafood Restaurant,Pizza Place,Fast Food Restaurant,Bar,Sushi Restaurant,Italian Restaurant,Scenic Lookout,Food,Sporting Goods Shop
3,Lafayette,Sandwich Place,Bar,Cajun / Creole Restaurant,Fast Food Restaurant,Gas Station,Café,Music Venue,Pizza Place,Bank,Hotel
4,Lake Charles,American Restaurant,Pharmacy,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Museum,Bar,Grocery Store,Paper / Office Supplies Store,Pub
5,Metairie,Grocery Store,Sushi Restaurant,Bar,Seafood Restaurant,Burger Joint,Smoothie Shop,Italian Restaurant,Sandwich Place,Salon / Barbershop,Spa
6,New Orleans,Hotel,Cajun / Creole Restaurant,Seafood Restaurant,Italian Restaurant,Hotel Bar,Music Venue,Bar,Coffee Shop,Cocktail Bar,French Restaurant
7,Shreveport,Bar,Clothing Store,Hotel,Fast Food Restaurant,American Restaurant,Steakhouse,Sporting Goods Shop,Seafood Restaurant,Burger Joint,Casino


In [66]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters, 5 apparent centers of data in the state (NW,New Orleans, South Central, Baton Rouge, SW)
kclusters = 5

LA_grouped_clustering = LA_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(LA_grouped_clustering)

# add clustering labels
city_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

LA_merged = dfLAtop8

# merge LA_grouped with dfLAtop10 to add latitude/longitude for each city
LA_merged = LA_merged.join(city_venues_sorted.set_index('City'), on='City')

LA_merged.head() # check the last columns!

,Rank,City,Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,New Orleans,390144,29.949932,-90.070116,2,Hotel,Cajun / Creole Restaurant,Seafood Restaurant,Italian Restaurant,Hotel Bar,Music Venue,Bar,Coffee Shop,Cocktail Bar,French Restaurant
1,2,Baton Rouge,220236,30.445960,-91.187380,4,Museum,Bar,Hotel,American Restaurant,Historic Site,Park,Performing Arts Venue,Music Venue,History Museum,Convenience Store
2,3,Shreveport,187112,32.522183,-93.765194,4,Bar,Clothing Store,Hotel,Fast Food Restaurant,American Restaurant,Steakhouse,Sporting Goods Shop,Seafood Restaurant,Burger Joint,Casino
3,4,Metairie,143481,29.989993,-90.150543,3,Grocery Store,Sushi Restaurant,Bar,Seafood Restaurant,Burger Joint,Smoothie Shop,Italian Restaurant,Sandwich Place,Salon / Barbershop,Spa
4,5,Lafayette,126185,30.224090,-92.019843,0,Sandwich Place,Bar,Cajun / Creole Restaurant,Fast Food Restaurant,Gas Station,Café,Music Venue,Pizza Place,Bank,Hotel


In [67]:
# create map
map_clusters = folium.Map(location=[lat, long], zoom_start=8)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(LA_merged['Latitude'], LA_merged['Longitude'], LA_merged['City'], LA_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [68]:
LA_merged.sort_values(by=['Cluster Labels'])

,Rank,City,Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,5,Lafayette,126185,30.224090,-92.019843,0,Sandwich Place,Bar,Cajun / Creole Restaurant,Fast Food Restaurant,Gas Station,Café,Music Venue,Pizza Place,Bank,Hotel
5,6,Lake Charles,78396,30.226595,-93.217376,0,American Restaurant,Pharmacy,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Museum,Bar,Grocery Store,Paper / Office Supplies Store,Pub
7,8,Kenner,66340,29.994226,-90.241781,1,Coffee Shop,Seafood Restaurant,Pizza Place,Fast Food Restaurant,Bar,Sushi Restaurant,Italian Restaurant,Scenic Lookout,Food,Sporting Goods Shop
0,1,New Orleans,390144,29.949932,-90.070116,2,Hotel,Cajun / Creole Restaurant,Seafood Restaurant,Italian Restaurant,Hotel Bar,Music Venue,Bar,Coffee Shop,Cocktail Bar,French Restaurant
3,4,Metairie,143481,29.989993,-90.150543,3,Grocery Store,Sushi Restaurant,Bar,Seafood Restaurant,Burger Joint,Smoothie Shop,Italian Restaurant,Sandwich Place,Salon / Barbershop,Spa
6,7,Bossier City,68159,32.536936,-93.695269,3,Fried Chicken Joint,Mexican Restaurant,Clothing Store,Burger Joint,Fast Food Restaurant,Breakfast Spot,Steakhouse,Department Store,Discount Store,Japanese Restaurant
1,2,Baton Rouge,220236,30.445960,-91.187380,4,Museum,Bar,Hotel,American Restaurant,Historic Site,Park,Performing Arts Venue,Music Venue,History Museum,Convenience Store
2,3,Shreveport,187112,32.522183,-93.765194,4,Bar,Clothing Store,Hotel,Fast Food Restaurant,American Restaurant,Steakhouse,Sporting Goods Shop,Seafood Restaurant,Burger Joint,Casino
